# Data Generator (Saved in CSV)
   
Add values of $\alpha$, $\beta$, Length, Grid points, and Amplitude in the function.
The Wave Number is None for now

In [1]:
import pandas as pd
import numpy as np
from math import erf as erf_, sin, exp as exp_
from numpy import pi, sqrt
import numba as nb
from scipy.io import savemat

def DataGenerator(alpha, beta=0.1, L=6, t=500, N=200, amp=0, CSV=False):
    %run ./KFRequation_u.ipynb
    par = {
    'alpha': alpha,
    'beta': beta,
    'L': L,
    'N': N,
    'amp': amp
    }
    for wn in [None]:
        par['wn'] = wn
        tmax = 3000 if wn == 0.01 else t
        kfr_equation(tmax=tmax, **par)
        
    def loader(alpha, amp):
        _path = '../comp_res'
        fullpath = os.path.join(_path)
        filename = f'amp_{amp}' + f'alpha_{alpha}' 
        shock=np.load(os.path.join(filename + '.npz'))
        return shock
    
    shock = loader(par['alpha'], 0)
    @nb.vectorize(('f8(f8)'))
    def erf(x):
        return erf_(x)

    L=L
    N=N
    dx = L / N  # grid size

    x = np.empty(N + 7)
    x[0:3] = [(-L + dx * i) for i in [-3, -2, -1]]  # left ghost points
    x[3:N + 4] = [(-L + dx * i) for i in range(N + 1)]  # internal grid points on [-L,0]
    x[N + 4:N + 7] = np.arange(1, 4) * dx  # grid points ahead of the shock
    
    t=shock['t']
    u=shock['u']
    t=np.array([t])
    x=np.array([x])
    
    matDict = {"t": t.T, "x": x.T, "u": np.transpose(u)}
    
    if CSV==True:
        filename = f'amp_{amp}' + f'alpha_{alpha}'
        U_steadyState = u[:,-1] 
        pd.DataFrame(t).to_csv(filename + '_T.csv')
        pd.DataFrame(U_steadyState).to_csv(filename + '_U_steadyState.csv')
    else:
        filename = f'amp_{amp}' + f'alpha_{alpha}'
        savemat(filename + '.mat', matDict)
    return u.shape

In [7]:
u=DataGenerator(4.85, t=250) #DataGenerator(Alpha, Beta, L, t, N, Amp, ext)

<ipython-input-7-a5a72cc7fb6d>:1: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "set_bc" failed type inference due to: No implementation of function Function(<function ones_like at 0x0000024CD7A6D3A0>) found for signature:
 
 >>> ones_like(array(int64, 1d, A), dtype=Function(<class 'int'>))
 
There are 2 candidate implementations:
      - Of which 2 did not match due to:
      Overload of function 'ones_like': File: numba\core\typing\npydecl.py: Line 532.
        With argument(s): '(array(int64, 1d, A), dtype=Function(<class 'int'>))':
       No match.

During: resolving callee type: Function(<function ones_like at 0x0000024CD7A6D3A0>)
During: typing of call at <ipython-input-7-a5a72cc7fb6d> (17)


File "<ipython-input-7-a5a72cc7fb6d>", line 17:
def set_bc(x, Ym, Nleft, Nint, Nright):
    <source elided>

    Ym[Nleft] = Ym[Nint[0] * np.ones_like(Nleft, dtype=int)]
    ^

  @nb.jit('f8[:](f8[:], f8[:], i8[:], i8[:], i8[:])')
C:\User

amp_0alpha_4.85 

<ipython-input-7-689c77b4b0d8>:106: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "RK3" failed type inference due to: Invalid use of type(CPUDispatcher(<function weno5 at 0x0000024CE1969E50>)) with parameters (array(float64, 1d, C), readonly array(int64, 1d, C), Literal[int](203), Literal[int](0), none, none, Literal[int](1))
Known signatures:
 * (array(float64, 1d, A), array(int64, 1d, A), int64, float64, OptionalType(float64), OptionalType(float64), int64) -> array(float64, 1d, A)
During: resolving callee type: type(CPUDispatcher(<function weno5 at 0x0000024CE1969E50>))
During: typing of call at <ipython-input-7-689c77b4b0d8> (108)


File "<ipython-input-7-689c77b4b0d8>", line 108:
    def RK3(u, Source, x_lab, dt):
        Flux = weno5(u, Nweno, ishock, amp, x_lab, wn, sign)
        ^

  @nb.jit
C:\Users\SZF\anaconda3\envs\GPUtensorflow\lib\site-packages\numba\core\object_mode_passes.py:151: NumbaWarning: Function "RK3" was compi

=-=-=-=-=-= 12.7s
